In [0]:
#Importamos librerías
import numpy as np
import pandas as pd
import io
import requests
from sklearn.feature_extraction.text import TfidfVectorizer  #Funcion a investigar
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression   #Funcion de Regresión logistica
from sklearn.model_selection import cross_val_score  #Función de cross validation
from sklearn.pipeline import make_union  


In [0]:
#Definimos los url para cargar los datos
url_train = "https://github.com/czammar/MA_finalproject/blob/master/data/train.csv?raw=true"
url_test = "https://raw.githubusercontent.com/czammar/MA_finalproject/master/data/test.csv?raw=true"
url_test_labels = "https://github.com/czammar/MA_finalproject/blob/master/data/test_labels.csv?raw=true"

In [0]:
# Leemos los los datos de train, test y labels del test desde nuestro repositorio usando requesto con io
rtrain=requests.get(url_train).content
rtest=requests.get(url_test).content
rtest_labels=requests.get(url_test_labels).content

In [0]:
#Define los clasificaciones de toxicidad
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [0]:
!ls

sample_data


In [0]:
# Cargamos io hacia pandas
train = pd.read_csv(io.StringIO(rtrain.decode('utf-8')))
test = pd.read_csv(io.StringIO(rtest.decode('utf-8')))

In [0]:
test.head(5)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [0]:
train.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0


In [0]:
#Se seleccionan las columnas de textos tanto de train como de test
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])  #Aqui se concatenan

In [0]:
all_text.head(2)

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
Name: comment_text, dtype: object

# Función TfidVectorizer de sklearn

In [0]:
# Funcion TfidVectorizer por palabra
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,   
    strip_accents='unicode', 
    analyzer='word',  
    token_pattern=r'\w{1,}',  
    ngram_range=(1, 1),  
    max_features=30000) 
# Funcion TfidVectorizer por caracter
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)

TfidfVectorizer es una función que converte una colección de documentos en bruto en una matriz de características TF-IDF.
Recibe los siguientes argumentos: <br>
1) sublinear_tf : Esta entrada es de tipo *Boolean* y aplica una escala del tf ( Donde Tf se refiere a la frecuencia del termino) 1+log(tf) <br>
2) strip_accents puede recibir 3 parámetros : {‘ascii’, ‘unicode’, None}  es para remover acentos, la forma más rápida es con *ascii* <br>
3) analyzer- Es de tipo *String* y puede recibir los siguientes argumentos : {‘word’, ‘char’, ‘char_wb’} Indica en que se debe aplicar la característica, pj a un caracter o a una palabra. <br>
4) token_pattern - Denota que constituye el token, sólo utilizar si analyzer == 'word' <br>
5) ngram_range - Indica el rango superior e inferior de los unigramas, pj (1,1) significa que son unigramas. <br>
5) max_features - Puede ser *Int* o *None*, puede crear un vocabulario que solo considere las características máximas superiores ordenadas por frecuencia de término en todo el corpus.


In [0]:
#Union de los transformadores especificados arriba
vectorizer = make_union(word_vectorizer, char_vectorizer, n_jobs=2)

In [0]:
vectorizer.fit(all_text)  #Esta función se encarga de vectoriza el texto
train_features = vectorizer.transform(train_text)
test_features = vectorizer.transform(test_text)

In [0]:
first_vector_tfidfvectorizer=test_features[1]  #test
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
13115,0.541680
22087,0.483481
10311,0.414268
26182,0.376980
13957,0.228239
...,...
20007,0.000000
20008,0.000000
20009,0.000000
20010,0.000000


La función TfidfVectorizer calcula la importancia de cada palabra de acuerdo a su extrañesa en el texto, es decir si la palabra se repite mucho como por ejemplo : "you" tiene poca importancia porque aparece muchas veces en los textos, pero la palabra " succesful" tendrá un mayor score porque casi no es utilizada. Es un tipo de medición de *key words*.

In [0]:
#Ejemplo
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(test_text)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[1]
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
imo,0.533813
rfc,0.475241
fine,0.409552
title,0.372927
is,0.269062
...,...
gnats,0.000000
gnatures,0.000000
gnauk,0.000000
gnawed,0.000000


Los resultados no son identicos porque se le introdujeron más parámetros a las funciones, pero la idea general es la misma, buscar al importancia de las palabras que son poco frencuentes, considerando la frecuencia del término y la frecuencia inversa del documento.

# Aplicación Regresión Logística

In [0]:
scores = []  
submission = pd.DataFrame.from_dict({'id': test['id']}) 
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(
        classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9787977659958528
CV score for class severe_toxic is 0.98877656229063
CV score for class obscene is 0.9903280439557607
CV score for class threat is 0.9887839859212525
CV score for class insult is 0.9828850848233746
CV score for class identity_hate is 0.9831386353878381
Total CV score is 0.9854516797291181


1) Indica el data frame para ingresar los datos de validación y se aplica un ciclo for para cada una de las clasificaicones de toxicidad que se tienen.<br>
2) Se aplica el modelo de *Rregresión logística* sobre la variable target *clasifiacion*. El método solver con el que esta aplicando regresión logística es 'sag'(Stochastic Average Gradient descent) que funciona bien para grandes conjuntos de datos.
3) Se evalua el score con cross_validation con el scoring de la *ROC*. <br>
4) Imprime para cada clasifiación de toxicidad su score de acuerdo al modelo implementado.<br>
5) Calcula la probabilidad en la que cada comentario incurriria dividido por clases de toxicidad, estos resultados se guarda en el archivo submission.csv 


In [0]:
#submission.to_csv('submission.csv', index=False)

Referencias <br>
* https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.Xqz2zFMzboz <br>
* https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html <br>
* https://www.kaggle.com/thousandvoices/logistic-regression-with-words-and-char-n-grams/code <br>